<a href="https://colab.research.google.com/github/marcos-da-luz-pinto/TSFMs/blob/main/AutoARIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pmdarima pandas numpy scipy matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 689.1/689.1 kB 10.7 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from pmdarima import auto_arima
from scipy.stats import norm
import warnings

warnings.filterwarnings("ignore")

# 1. Carregar dados
df = pd.read_excel('data.xlsx')
df['Data'] = pd.to_datetime(df['Data'])
df = df.sort_values('Data').set_index('Data')

# Variáveis para análise
colunas_alvo = ['INCC- DI_var_mes', 'Taxa_SELIC_avg', 'CUB Global', 'Cub Material', 'CUB Mão de Obra', 'CUB Desp. Administrativa', 'CUB Equipamento']
contexts = [24, 60, 120]
horizons = [12, 24]
n_windows = 10  # Número de repetições (janelas) por cenário, como no artigo

# Funções de métricas
def calculate_mase(y_true, y_pred, y_train, seasonal_period=12):
    denom = np.mean(np.abs(np.diff(y_train, seasonal_period)))
    if denom == 0: return np.nan
    return np.mean(np.abs(y_true - y_pred)) / denom

def calculate_crps_normal(y_true, mu, sigma):
    sigma = np.maximum(sigma, 1e-6)
    z = (y_true - mu) / sigma
    crps = sigma * (z * (2 * norm.cdf(z) - 1) + 2 * norm.pdf(z) - 1/np.sqrt(np.pi))
    return np.mean(crps)

results = []

for col in colunas_alvo:
    series = df[col].dropna()
    print(f"Processando {col}...")

    for C in contexts:
        for H in horizons:
            mase_list = []
            crps_list = []

            # Rolling Origin: retrocede no tempo para criar n_windows amostras
            # Começamos do fim da série e voltamos 'step' meses a cada iteração
            for w in range(n_windows):
                end_idx = len(series) - w
                start_idx = end_idx - (C + H)

                if start_idx < 0: break # Para se não houver dados suficientes

                # Divisão da Janela Atual
                window_data = series.iloc[start_idx:end_idx]
                train = window_data.iloc[:C]
                test = window_data.iloc[C:]

                try:
                    # Ajuste do modelo (SARIMA)
                    model = auto_arima(train, seasonal=True, m=12,
                                       seasonal_test='ch',
                                       error_action='ignore',
                                       suppress_warnings=True)

                    forecast_mean, conf_int = model.predict(n_periods=H, return_conf_int=True)
                    forecast_sigma = (conf_int[:, 1] - conf_int[:, 0]) / (2 * 1.96)

                    mase_list.append(calculate_mase(test.values, forecast_mean.values, train.values))
                    crps_list.append(calculate_crps_normal(test.values, forecast_mean.values, forecast_sigma))
                except:
                    continue

            if mase_list:
                results.append({
                    'Variável': col,
                    'C': C, 'H': H,
                    'MASE (Mediana)': round(np.median(mase_list), 4),
                    'CRPS (Mediana)': round(np.median(crps_list), 4),
                    'Amostras (n)': len(mase_list)
                })

# Gerar Tabela Final
results_df = pd.DataFrame(results)
print("\n--- RESULTADOS AutoARIMA ---")
print(results_df.to_string(index=False))

# Salvar resultados em csv
results_df.to_csv('RESULTADOS AutoARIMA', index=False)

Processando INCC- DI_var_mes...
Processando Taxa_SELIC_avg...
Processando CUB Global...
Processando Cub Material...
Processando CUB Mão de Obra...
Processando CUB Desp. Administrativa...
Processando CUB Equipamento...

--- RESULTADOS AutoARIMA ---
                Variável   C  H  MASE (Mediana)  CRPS (Mediana)  Amostras (n)
        INCC- DI_var_mes  24 12          0.0009          0.2362            10
        INCC- DI_var_mes  24 24          0.0021          0.4021            10
        INCC- DI_var_mes  60 12          0.0008          0.1715            10
        INCC- DI_var_mes  60 24          0.0018          0.2838            10
        INCC- DI_var_mes 120 12          0.0007          0.1754            10
        INCC- DI_var_mes 120 24          0.0006          0.1811            10
          Taxa_SELIC_avg  24 12          0.0220          1.0522            10
          Taxa_SELIC_avg  24 24          0.0378          5.8357            10
          Taxa_SELIC_avg  60 12          0.0085   